In [1]:
import pandas as pd
import numpy as np
import dateutil.tz
import datetime as dt
import json

from io import StringIO
import urllib3
import logging
import sys
from json import dumps
import time

def get_ymd(datetime):
    year = datetime.year
    month = datetime.month
    day = datetime.day
            
    if month < 10:
        month = '0' + str(month)
    if day < 10:
        day = '0' + str(day)
    return year, month, day

def first_day_next_month(date):
    return (date.replace(day=1) + dt.timedelta(days=32)).replace(day=1)

def last_second_of_month(date: str) -> str:
    return str((pd.event_timestamp(date) + pd.offsets.MonthEnd(0)).date()) + " 23:59:59"

def first_second_of_month(date: str) -> str:
    return str((pd.event_timestamp(date) + pd.offsets.MonthBegin(0)).date()) + " 00:00:00"

streamer = StringIO()

def setup_logging():
    logger = logging.getLogger()
    for h in logger.handlers:
        logger.removeHandler(h)
     
    h = logging.StreamHandler(stream = streamer)
    h.setFormatter(logging.Formatter("%(asctime)s %(levelname)s: %(message)s",
                              "%Y-%m-%d %H:%M:%S"))
    logger.addHandler(h)
    logger.setLevel(logging.INFO)
    return logger

def query_log(query_id, table, logger):
    status = wr.athena.get_query_execution(query_id)['Status']['State']
    if wr.athena.get_query_execution(query_id)['Status']['State'] in ['FAILED', 'CANCELLED']:
        logger.critical(table + ': query is in ' + status + ' State. ' + 'QueryID: ' + query_id)
    else:
        logger.info(table + ': query is in ' + status + ' State. ' + 'QueryID: ' + query_id)
    return None


In [2]:
# !python -m pip install amazon-textract-caller --upgrade
# !python -m pip install amazon-textract-response-parser --upgrade

In [3]:
# !pip install 

In [4]:
import re
bool(re.match('^[a-zA-Z]', ''))

False

# Static Raw Feature 

In [5]:
def get_cibil_score(cibil_info):
    for item in cibil_info:
        if item.isnumeric():
            return item
    return ''

In [6]:
def get_static_raw_cibil_features(df):
    dfs=[]
    for idx,row in df.iterrows():
        
        user_id = row.name
        gender = row['gender']
        name = row['name']
        total_email = len(row['email'])
        total_phone_nos = len(row['phone_no'])
        dob = row['dob']
        user_identifier = row['user_identifier']
        # remove from startic in final version
#         age = get_age(date(int(dob_list[2]),int(dob_list[1]),int(dob_list[0])))
        cibil_score = get_cibil_score(row['cibil_info_with_factors'])
        total_address = len(row['address'])
        total_loans = len(row['account_info'])

        df_static = pd.DataFrame({ 'user_id' : [user_id], 'name':[name],'gender':[gender], 'total_email':[total_email],'dob':[dob],'cibil_score':[cibil_score],
                       'total_address':[total_address],'total_loans':[total_loans],'total_phone_nos': [total_phone_nos], 'user_identifier':[user_identifier] })
        dfs.append(df_static)
    return pd.concat(dfs)
    

In [7]:
! ls ../cibil_data/train/

cibil_out  cibil_pdf  errors.csv  new_cibil_out  new_cibil_pdf


In [9]:
import glob 
all_csv = glob.glob('../cibil_data/train/cibil_out/*.csv') + glob.glob('../cibil_data/train/new_cibil_out/*.csv')

In [10]:
# all_csv

In [11]:
dfs = []
for csv in all_csv:
    dfs.append(pd.read_csv(csv))
dfs = pd.concat(dfs)

In [12]:
dfs.reset_index(inplace=True)

In [13]:
dfs.columns

Index(['index', 'name', 'gender', 'cibil_info_with_factors', 'cibil_name',
       'user_identifier', 'dob', 'address', 'phone_no', 'email', 'acc_summary',
       'account_info', 'enquiry', 'account_info_new'],
      dtype='object')

In [14]:
change_cols = ['cibil_info_with_factors', 'address', 'phone_no', 'email', 'acc_summary', 'account_info','enquiry','account_info_new']

In [15]:
for col in change_cols:
    dfs[col] = dfs[col].apply(lambda x:eval(x))

In [16]:
# dfs['enquiry'][10]

In [17]:
df_static = get_static_raw_cibil_features(dfs)

In [18]:
df_static

,user_id,name,gender,total_email,dob,cibil_score,total_address,total_loans,total_phone_nos,user_identifier
0,0,SHAIK HUSAN PASHA S/O SHAIK,MALE,4,02/02/1980,645,4,11,4,"{'PAN': 'HQPPS0603K', 'VOTER ID': 'YAV1651389'..."
0,1,GINAJALA VEERAJU POTHU RAJU,MALE,0,01/01/1966,,1,0,1,{'VOTER ID': 'IMZ1676123'}
0,2,KORRA RAVI,MALE,0,02/04/1980,712,4,1,2,"{'PAN': 'BUKPK3225N', 'RATION CARD': '12357056..."
0,3,SANAGAPALLI SARADHA,FEMALE,1,21/07/1993,757,3,7,4,"{'PAN': 'FMEPS5341P', 'VOTER ID': 'ZAF0660556'..."
0,4,MEKA SRINU,MALE,0,01/01/1987,747,2,1,1,"{'PAN': 'HPOPM6548R', 'VOTER ID': 'UDD0141804'}"
...,...,...,...,...,...,...,...,...,...,...
0,1311,VINOREXLINE AR,FEMALE,4,05/06/1991,738,4,17,4,"{'PAN': 'CIJPV7471D', 'VOTER ID': 'TRQ0365726'..."
0,1312,SAGAR VASANTRAO LANDGE LANDGE,MALE,0,01/06/1986,694,4,7,4,"{'PAN': 'ADMPL4711E', 'DRIVING LICENSE': 'MH12..."
0,1313,RANKIREDDY NAGESWARA RAO,MALE,2,02/04/1976,654,2,8,4,"{'PAN': 'DUUPR3358M', 'UNIVERSAL ID': '3674329..."
0,1314,PANTHAM KRISHNA,MALE,0,18/05/1990,521,2,1,3,"{'PAN': 'DPWPP4740C', 'UNIVERSAL ID': '4037596..."


## DPD Table

In [19]:
# df_nested_list['AccountInformation']

   - proper case/form for overdue calculation for all type of loans 
   - STD ~0 or not for v0 ? 
   - skip overdue for v0 
           -- schedule pmt 

In [20]:
# (df_nested_list['AccountInformation'][0][0]['AMOUNTS']['SanctionedAmount'])

In [21]:
def get_timestamp(dpd_list):
    final_json = {}
    if (len(dpd_list)) > 0:
        
        yrs = list(dpd_list.keys())[::-1]
        
        for item in yrs:
            dpd_elem = dpd_list[item]
            for k,v in dpd_elem.items():
    #             print(k,v)
                final_json[k+'-'+item]=v
    return final_json
    

In [22]:
df_static.columns

Index(['user_id', 'name', 'gender', 'total_email', 'dob', 'cibil_score',
       'total_address', 'total_loans', 'total_phone_nos', 'user_identifier'],
      dtype='object')

In [23]:
# dfs

In [24]:
for k,v in (dfs['account_info'][0][0]['DPD_INFO']).items():
    print(pd.to_datetime(k,format='%m-%y'))

2022-07-01 00:00:00
2022-06-01 00:00:00
2022-05-01 00:00:00
2022-04-01 00:00:00
2022-03-01 00:00:00
2022-02-01 00:00:00
2022-01-01 00:00:00
2021-12-01 00:00:00
2021-11-01 00:00:00
2021-10-01 00:00:00
2021-09-01 00:00:00
2021-08-01 00:00:00
2021-07-01 00:00:00


In [25]:
def get_dpd_raw_table(df):
    dfs = [] 
    for idx,row in df.iterrows():
        user_id = row.name 
        acc_info_list = row['account_info']
        dpd_info_json = row['account_info_new']
        for i in range(len(acc_info_list)):
            loan_id = i 
            loan_info = acc_info_list[i]
            sanc_amount =''
            loan_type = loan_info['ACCOUNT']['TYPE']
            ownership = loan_info['ACCOUNT']['ownership']
            if 'sanctioned' in loan_info['AMOUNTS']:
                sanc_amount=loan_info['AMOUNTS']['sanctioned']
            interest_rate = loan_info['AMOUNTS']['interest_rate']
            repayment_tenure = loan_info['AMOUNTS']['repay_tenure']
            emi_amount = loan_info['AMOUNTS']['emi']
            pmt_freq = loan_info['AMOUNTS']['pmt_freq']
            open_date = loan_info['DATES']['opened']
            closed_date = loan_info['DATES']['closed']
#             dpd_list = loan_info['PaymentHistory']['dayPayDue']
            timestamp_json = dpd_info_json[i]
            for k,v in timestamp_json.items():
                is_open=1
                is_closed=0
                ## dd-mm-yyyy
                
                if len(closed_date.split('-'))>=3:
#                     print(closed_date)
                    try:
        
                        curr_timestamp = pd.to_datetime(k,format='%m-%y')
                        closed_date_m_y = pd.to_datetime(closed_date,dayfirst=True)
                        if curr_timestamp.year==closed_date_m_y.year and curr_timestamp.month==closed_date_m_y.month:
                            is_open=0
                            is_closed=1
                        df_dpd = pd.DataFrame({'user_id':[user_id],'loan_id':[loan_id],'timestamp':[k],'dpd':[v],'loan_type':[loan_type],'ownership':[ownership],'sanc_amount':[sanc_amount], 'interest_rate':[interest_rate], 
                                      'repayment_tenure':[repayment_tenure],'emi_amount':[emi_amount],'pmt_freq':[pmt_freq],'open_date':[open_date],
                                      'closed_date':[closed_date],'is_open':[is_open],'is_closed':[is_closed]})
                        dfs.append(df_dpd)
                    except:
                        continue
                

                    
                        
    return pd.concat(dfs)
            

In [26]:
df_raw_dpd = get_dpd_raw_table(dfs)

In [27]:
# df_raw_dpd

In [28]:
 df_raw_dpd['timestamp_new'] = df_raw_dpd['timestamp'].apply(lambda x:pd.to_datetime(x,format='%m-%y'))


In [29]:
# df_raw_dpd['dpd'].value_counts()

In [30]:
def post_process_raw_dpd(df):
#     df['next_loan_id'] = df['loan_id'].shift(-1)

    df['next_user_id'] = df['user_id'].shift(-1)
#     df['next_loan_id'] = df['next_loan_id'].fillna(0)
    df['next_user_id'] = df['next_user_id'].fillna(0)
#     df['next_loan_id'] = df['next_loan_id'].apply(lambda x: int(x)  )
    df['next_user_id'] = df['next_user_id'].apply(lambda x: int(x) )

    return df

In [31]:
df_raw_dpd_new = post_process_raw_dpd(df_raw_dpd)

In [32]:
df_raw_dpd_new.shape

(96707, 17)

In [194]:
df_raw_dpd_new.user_id.nunique()

847

In [33]:
# df_raw_dpd_new[:35]

In [34]:
df_raw_dpd_new['timestamp_new'].min()

Timestamp('2003-08-01 00:00:00')

In [35]:
df_raw_dpd_new.sort_values(['user_id','timestamp_new'],inplace=True)

In [36]:
df_raw_dpd_new.columns

Index(['user_id', 'loan_id', 'timestamp', 'dpd', 'loan_type', 'ownership',
       'sanc_amount', 'interest_rate', 'repayment_tenure', 'emi_amount',
       'pmt_freq', 'open_date', 'closed_date', 'is_open', 'is_closed',
       'timestamp_new', 'next_user_id'],
      dtype='object')

In [37]:
# df_raw_dpd_new.reset_index(inplace=True)

In [38]:
df_raw_dpd_new[['timestamp', 'dpd', 'loan_type', 'ownership',
       'sanc_amount', 'interest_rate', 'repayment_tenure', 'emi_amount','open_date', 'closed_date']][37:48]

,timestamp,dpd,loan_type,ownership,sanc_amount,interest_rate,repayment_tenure,emi_amount,open_date,closed_date
0,4-19,000,Two-wheeler Loan,INDIVIDUAL,,,30.00,3500.00,29-10-2018,12-05-2021
0,5-19,000,Two-wheeler Loan,INDIVIDUAL,,,30.00,3500.00,29-10-2018,12-05-2021
0,6-19,000,Two-wheeler Loan,INDIVIDUAL,,,30.00,3500.00,29-10-2018,12-05-2021
0,7-19,000,Two-wheeler Loan,INDIVIDUAL,,,30.00,3500.00,29-10-2018,12-05-2021
0,8-19,000,Two-wheeler Loan,INDIVIDUAL,,,30.00,3500.00,29-10-2018,12-05-2021
0,9-19,000,Two-wheeler Loan,INDIVIDUAL,,,30.00,3500.00,29-10-2018,12-05-2021
0,10-19,000,Two-wheeler Loan,INDIVIDUAL,,,30.00,3500.00,29-10-2018,12-05-2021
0,11-19,000,Two-wheeler Loan,INDIVIDUAL,,,30.00,3500.00,29-10-2018,12-05-2021
0,12-19,000,Two-wheeler Loan,INDIVIDUAL,,,30.00,3500.00,29-10-2018,12-05-2021
0,1-20,000,Two-wheeler Loan,INDIVIDUAL,,,30.00,3500.00,29-10-2018,12-05-2021


In [39]:
# count=0
# for i,row in df_raw_dpd_new.iterrows():
#     try:
#         a = int(row['sanc_amount'])
#     except:
#         print(i)
#         count+=1
# print(count)

In [40]:
## Steps
## fill NA with mean values corresponding to the feature columns 
## preapre dummy overdue for each u,l pair 
##  double check the is_open and is_closed ! seems wrong for now 
## -> prepare overdue feature based on other attributes 

In [41]:
# float('3013353')

In [42]:
def get_sanc_amt(sanc_string):
    if sanc_string=='' or sanc_string is None:
        return 0
    sanc_elems =sanc_string.split(',')
    final_amt=''
    for item in sanc_elems:
        final_amt+=item
    return eval(final_amt)

In [43]:
get_sanc_amt('3,123,234')

3123234

In [44]:
 df_raw_dpd_new['sanc_amount_temp'] =  df_raw_dpd_new['sanc_amount'].apply(lambda x: get_sanc_amt(x))

In [45]:
# df_raw_dpd_new['interest_rate_temp'] = df_raw_dpd_new['interest_rate'].apply(lambda x: eval(x) if (x!='-' and  x!= None) else 0)


In [46]:
# df_raw_dpd_new['emi_temp'] = df_raw_dpd_new['emi_amount'].apply(lambda x: eval(x) if (x!='-' and  x!= None) else 0)


In [47]:
sanc_amt_loan_type_mean_df = df_raw_dpd_new.groupby('loan_type').agg({'sanc_amount_temp':'mean'}).reset_index()

In [48]:
# interest_rate_loan_type_mean_df = df_raw_dpd_new.groupby('loan_type').agg({'interest_rate_temp':'mean'}).reset_index()

In [49]:
# emi_loan_type_mean_df = df_raw_dpd_new.groupby('loan_type').agg({'emi_temp':'mean'}).reset_index()

In [50]:
# set(df_raw_dpd_new[df_raw_dpd_new['repayment_tenure']=='-']['sanc_amount'].tolist()) #[['open_date','closed_date','emi_amount','interest_rate']]

In [51]:
# df_raw_dpd_new['closed_date'].max()

##  Enquiry Table 

In [52]:
# dfs['enquiry'][0]

    {'member': 'NOT DISCLOSED',
      'enquiry_date': '29-09-2022',
      'enquiry_purpose': 'Two-wheeler Loan',
      'enquiry_amt': '500,000'},

In [53]:
def get_enquiry_table(df): 
    dfs = []
    for idx,row in df.iterrows():
        user_id = row.name
        enq_info = row['enquiry']
        for elem in enq_info:
            date = elem['enquiry_date']
            enq_purpose = elem['enquiry_purpose']
            try:
                
                enq_amt = get_sanc_amt(elem['enquiry_amt'])
            except:
#                 print(elem)
                break
            df_enq = pd.DataFrame({'user_id':[user_id],'date':[date],'enq_purpose':[enq_purpose],'enq_amount':[enq_amt]})
            dfs.append(df_enq)
    return pd.concat(dfs)

In [54]:
df_enquiry = get_enquiry_table(dfs)

In [55]:
df_enquiry.head(2)

,user_id,date,enq_purpose,enq_amount
0,3,12-05-2022,Credit Card,20000
0,3,16-11-2021,Other,1


In [56]:
df_enquiry['timestamp'] = df_enquiry['date'].apply(lambda x: pd.to_datetime(x))

# Feature Table 

In [57]:
df_static.columns

Index(['user_id', 'name', 'gender', 'total_email', 'dob', 'cibil_score',
       'total_address', 'total_loans', 'total_phone_nos', 'user_identifier'],
      dtype='object')

In [58]:
df_static.head(2)

,user_id,name,gender,total_email,dob,cibil_score,total_address,total_loans,total_phone_nos,user_identifier
0,0,SHAIK HUSAN PASHA S/O SHAIK,MALE,4,02/02/1980,645,4,11,4,"{'PAN': 'HQPPS0603K', 'VOTER ID': 'YAV1651389'..."
0,1,GINAJALA VEERAJU POTHU RAJU,MALE,0,01/01/1966,,1,0,1,{'VOTER ID': 'IMZ1676123'}


In [59]:
df_static[df_static['user_id']==0]['total_email'].tolist()[0]

4

In [60]:
df_raw_dpd.columns

Index(['user_id', 'loan_id', 'timestamp', 'dpd', 'loan_type', 'ownership',
       'sanc_amount', 'interest_rate', 'repayment_tenure', 'emi_amount',
       'pmt_freq', 'open_date', 'closed_date', 'is_open', 'is_closed',
       'timestamp_new', 'next_user_id', 'sanc_amount_temp'],
      dtype='object')

    user_id/key	timestamp	total_email	open_loans	closed_loans	total_loans 	overdue	gender	age	enquiry_till_date	tot_enquiry_purpose	tot_enquiry_amount	DPD	interest_rate	tenure	sanctioned_amount	loan_type	ownership	score

In [61]:
def get_flags(acc_info):
    for acc in acc_info:
        dpd_list = acc['PaymentHistory']['dayPayDue']
        timestamp_json = get_timestamp(dpd_list)
        for k,v in timestamp_json.items():
            if v=='900':
                return True
    return False

In [62]:
# df_nested_list[df_nested_list['AccountInformation'].apply(lambda x: get_flags(x))]

In [63]:
# df_raw_dpd_new[df_raw_dpd_new['dpd']=='900']

In [65]:
df_raw_dpd_new['dpd'].value_counts()

000    54247
XXX    16611
STD    13359
030      741
026      419
       ...  
338        1
522        1
582        1
643        1
438        1
Name: dpd, Length: 674, dtype: int64

In [66]:
def fill_zero_val_from_loan_type(sanc_amount,loan_type):
    sanc_list =sanc_amt_loan_type_mean_df[sanc_amt_loan_type_mean_df['loan_type']==loan_type]['sanc_amount_temp'].tolist()
    if len(sanc_list)>0:  
        sanc_amt_loan_type = sanc_list[0]
        if int(sanc_amount)==0:
            return sanc_amt_loan_type
    return sanc_amount
    

In [67]:
df_raw_dpd_new['sanc_amt_final'] = df_raw_dpd_new.apply(lambda x:fill_zero_val_from_loan_type(x['sanc_amount_temp'],x['loan_type']),axis=1)

In [68]:
# df_raw_dpd_new[df_raw_dpd_new['sanc_amt_final']==0]

In [69]:
## use this for feature only 
def bucket_dpd(dpd_val):
    final_val=dpd_val
    if dpd_val=='STD' or dpd_val=='XXX':
        final_val=0  
    if dpd_val.isnumeric():
        dpd_val=int(dpd_val)
        if dpd_val<10:
            final_val=0
        elif dpd_val>=10 and dpd_val<20:
            final_val=1
        elif dpd_val>=20 and dpd_val<30:
            final_val=2
        elif dpd_val>=30 and dpd_val<40:
            final_val=3
        elif dpd_val>=40 and dpd_val<50:
            final_val=4
        elif dpd_val>=50 and dpd_val<60:
            final_val=5
        elif dpd_val>=60 and dpd_val<70:
            final_val=6
        elif dpd_val>=70 and dpd_val<80:
            final_val=7
        elif dpd_val>=80 and dpd_val<90:
            final_val=8
        else:
            final_val=9
    else:
        final_val=0
    return final_val
    
        

In [70]:
def modified_dpd(dpd_val):
    final_val=dpd_val
    if dpd_val=='STD' or dpd_val=='XXX':
        final_val=0  
    if dpd_val.isnumeric():
        dpd_val=int(dpd_val)
    else:
        final_val=0
    return int(final_val)

In [71]:
df_raw_dpd_new['modified_dpd'] = df_raw_dpd_new['dpd'].apply(modified_dpd)

In [72]:
# df_raw_dpd_new['modified_dpd'].dtype

In [73]:
df_raw_dpd_new['dpd_bucket'] = df_raw_dpd_new['dpd'].apply(bucket_dpd)

In [74]:
# df_raw_dpd_new['dpd_y'].value_counts()

In [75]:
# df_enquiry['enq_purpose'].nunique()

In [76]:
# df_enquiry[df_enquiry['timestamp']<'2014-01-01']

In [77]:
((pd.to_datetime('2022-10-1')- pd.to_datetime('1-2-2002')).days)/365

20.75890410958904

In [78]:
 (pd.date_range("2022-10-1", periods=3, freq="-1MS")).tolist()[1]

Timestamp('2022-09-01 00:00:00', freq='-1MS')

In [79]:
now = pd.to_datetime("2022-10-1")-    pd.Timedelta('1D')
last2 = now - pd.DateOffset(months=2)
last3 = now - pd.DateOffset(months=3)

In [80]:
# df_raw_dpd_new.query("(@now >= timestamp_new >= @last2) | (@now >= timestamp_new >= @last3)")

In [81]:
## 01-06-2022  (02-06-2022)
## 01-07-2022
## 01-08-2022
## 31-08-2022
## 01-09-2022
## month level date time range 

In [82]:
from datetime import date
 
def get_age(birthdate,curr_date):
    today = date.today()
    age = curr_date.year - birthdate.year - ((curr_date.month, curr_date.day) < (birthdate.month, birthdate.day))
    return age

In [83]:
## steps to eval all fileds : 
## make raw_dpd as central table as this contains all the info regarding user_id and loan_id
## user_id , key --> get from the raw_dpd_table
## for this pair eval all the fields  | for static directly query from static_raw_df | for enq perform all ops from enq_table 
## eval timestamp based on mm-yyyy from the dpd table to eval the dpd related features 
## static -> 'gender', 'total_email', 'dob', 'age', 'cibil_score',
##       'total_address', 'total_loans', 'total_phone_nos'
def get_cibil_feature_table(df_raw_static,df_raw_dpd,df_raw_enquiry):
    dfs = []
    for idx,row in df_raw_dpd.iterrows():
        key = str(row['user_id'])+'-'+str(row['loan_id'])
        new_df = df_raw_dpd[(df_raw_dpd['user_id']==row['user_id']) & (df_raw_dpd['loan_id']==row['loan_id'])]
        timestamp = row['timestamp']
        timestamp_new = row['timestamp_new']
        prev_day = timestamp_new -  pd.Timedelta('1D')
        next_day = timestamp_new +  pd.Timedelta('1D')
        total_email = df_raw_static[df_raw_static['user_id']==row['user_id']]['total_email'].tolist()[0]
        gender = df_raw_static[df_raw_static['user_id']==row['user_id']]['gender'].tolist()[0]
        dob = df_raw_static[df_raw_static['user_id']==row['user_id']]['dob'].tolist()[0]
        age = get_age(pd.to_datetime(dob),timestamp_new)
#         age = df_raw_static[df_raw_static['user_id']==row['user_id']]['age'].tolist()[0]
        cibil_score = df_raw_static[df_raw_static['user_id']==row['user_id']]['cibil_score'].tolist()[0]
        total_address = df_raw_static[df_raw_static['user_id']==row['user_id']]['total_address'].tolist()[0]
#         total_loans = df_raw_static[df_raw_static['user_id']==row['user_id']]['total_loans'].tolist()[0]
        total_phone_nos = df_raw_static[df_raw_static['user_id']==row['user_id']]['total_phone_nos'].tolist()[0]

        total_loans = len(set(df_raw_dpd[(df_raw_dpd['user_id']==row['user_id']) & (df_raw_dpd['timestamp_new']<=row['timestamp_new'])
                               &(df_raw_dpd['is_open']==1)]['loan_id'].tolist()))
        closed_loans = len(set(df_raw_dpd[(df_raw_dpd['user_id']==row['user_id']) & (df_raw_dpd['timestamp_new']<=row['timestamp_new'])
                               &(df_raw_dpd['is_closed']==1)]['loan_id'].tolist()))
    
        open_loans = total_loans - closed_loans
        dpd_bucket = row['dpd_bucket']
        dpd_provided = row['dpd']
        sanc_amount = new_df['sanc_amt_final'].tolist()[0]
        ## mm-yyyy 
        
        last_3_months_dpd= sum(new_df[(new_df['timestamp_new'].apply(lambda x: x <timestamp_new and x>=(timestamp_new-pd.DateOffset(months=3))))]['modified_dpd'].tolist())
        
        #last_3_months_dpd =  sum(new_df[(new_df['timestamp_new'].apply(lambda x: x in (pd.date_range(prev_day, periods=95, freq="-1D"))))]['modified_dpd'].tolist())
        last_6_months_dpd = sum(new_df[(new_df['timestamp_new'].apply(lambda x: x <timestamp_new and x>=(timestamp_new-pd.DateOffset(months=6))))]['modified_dpd'].tolist())
        last_12_months_dpd = sum(new_df[(new_df['timestamp_new'].apply(lambda x: x <timestamp_new and x>=(timestamp_new-pd.DateOffset(months=12))))]['modified_dpd'].tolist())
        last_36_months_dpd = sum(new_df[(new_df['timestamp_new'].apply(lambda x: x <timestamp_new and x>=(timestamp_new-pd.DateOffset(months=36))))]['modified_dpd'].tolist())
        
        ## next month summation (mm-yyyy, dd-mm-yyyy)
        next_3_months_dpd = sum(new_df[(new_df['timestamp_new'].apply(lambda x: x >timestamp_new and x<=(timestamp_new+pd.DateOffset(months=3))))]['modified_dpd'].tolist())
        next_6_months_dpd = sum(new_df[(new_df['timestamp_new'].apply(lambda x: x >timestamp_new and x<=(timestamp_new+pd.DateOffset(months=6))))]['modified_dpd'].tolist())
        next_12_months_dpd = sum(new_df[(new_df['timestamp_new'].apply(lambda x:x >timestamp_new and x<=(timestamp_new+pd.DateOffset(months=12))))]['modified_dpd'].tolist())
        next_36_months_dpd = sum(new_df[(new_df['timestamp_new'].apply(lambda x: x >timestamp_new and x<=(timestamp_new+pd.DateOffset(months=36))))]['modified_dpd'].tolist())
        ## enquiry level features -> total_enq_till_date  | unique_enquiry_purpose  | total_enq_amt 
        total_enq_till_date = df_raw_enquiry[(df_raw_enquiry['user_id']==row['user_id']) & (df_raw_enquiry['timestamp']<timestamp_new)].shape[0]
        unique_enquiry_purpose = df_raw_enquiry[(df_raw_enquiry['user_id']==row['user_id']) & (df_raw_enquiry['timestamp']<timestamp_new)]['enq_purpose'].nunique()
        total_enq_amt = df_raw_enquiry[(df_raw_enquiry['user_id']==row['user_id']) & (df_raw_enquiry['timestamp']<timestamp_new)]['enq_amount'].sum()
        
        df_feature = pd.DataFrame({
            'key':[key] , 'timestamp':[timestamp] ,'datetime_formatted' : [timestamp_new], 'cibil_score' : [cibil_score], 
            'total_email' : [total_email], 'gender' : [gender], 'age' : [age] ,'open_loans':[open_loans],'closed_loans':[closed_loans] ,  'total_address' : [total_address],
            'sanc_amount':[sanc_amount],'total_loans' : [total_loans], 'total_phone_nos' : [total_phone_nos], 'dpd':[dpd_provided],'dpd_bucket':[dpd_bucket],
            'last_3_months_dpd' :[last_3_months_dpd],'last_6_months_dpd' : [last_6_months_dpd],'last_12_months_dpd':[last_12_months_dpd],'last_36_months_dpd':[last_36_months_dpd],
            'next_3_months_dpd' : [next_3_months_dpd],'next_6_months_dpd':[next_6_months_dpd] , 'next_12_months_dpd':[next_12_months_dpd], 'next_36_months_dpd':[next_36_months_dpd],
            'total_enq_till_date':[total_enq_till_date],'unique_enquiry_purpose':[unique_enquiry_purpose],'total_enq_amt':[total_enq_amt]
            
        })
        dfs.append(df_feature)
    return pd.concat(dfs)
        
        
    

In [84]:
# 

In [85]:
df_cibil_feature =  get_cibil_feature_table(df_static,df_raw_dpd_new,df_enquiry)

In [86]:
df_cibil_feature

,key,timestamp,datetime_formatted,cibil_score,total_email,gender,age,open_loans,closed_loans,total_address,...,last_6_months_dpd,last_12_months_dpd,last_36_months_dpd,next_3_months_dpd,next_6_months_dpd,next_12_months_dpd,next_36_months_dpd,total_enq_till_date,unique_enquiry_purpose,total_enq_amt
0,0-10,11-16,2016-11-01,645,4,MALE,36,1,0,4,...,0,0,0,0,0,0,0,0,0,0
0,0-10,12-16,2016-12-01,645,4,MALE,36,1,0,4,...,0,0,0,0,0,0,0,0,0,0
0,0-10,1-17,2017-01-01,645,4,MALE,36,1,0,4,...,0,0,0,0,0,0,0,0,0,0
0,0-10,2-17,2017-02-01,645,4,MALE,36,1,0,4,...,0,0,0,0,0,0,0,0,0,0
0,0-10,3-17,2017-03-01,645,4,MALE,37,1,0,4,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,1315-0,12-21,2021-12-01,658,0,MALE,42,1,0,3,...,29,29,35,90,90,90,90,0,0,0
0,1315-0,1-22,2022-01-01,658,0,MALE,43,0,1,3,...,90,90,96,0,0,0,0,0,0,0
0,1315-0,2-22,2022-02-01,658,0,MALE,43,0,1,3,...,180,180,186,0,0,0,0,0,0,0
0,1315-0,3-22,2022-03-01,658,0,MALE,43,0,1,3,...,180,180,186,0,0,0,0,0,0,0


In [87]:
df_cibil_feature.to_csv('dw_out/cibil_features_pdf_v1.csv',index=False)

In [88]:
df_cibil_feature = pd.read_csv('dw_out/cibil_features_pdf_v1.csv')

# Next Steps 
       for a given user loan type and timestamp_new  
  -- groupby user and loan_type and timestamp_new 
  
  -- replicate same for enquiry / enquiry_type 
   --join these two at uid level
   

In [89]:
df_cibil_feature['user_id'] = df_cibil_feature['key'].apply(lambda x:int(x.split('-')[0]))
df_cibil_feature['loan_id'] = df_cibil_feature['key'].apply(lambda x:int(x.split('-')[1]))


In [90]:
# just for curr use case | not to use this method for prod 
# next step change the dpd level feats 
# df

In [91]:
df_cibil_feature.tail(20)

,key,timestamp,datetime_formatted,cibil_score,total_email,gender,age,open_loans,closed_loans,total_address,...,last_36_months_dpd,next_3_months_dpd,next_6_months_dpd,next_12_months_dpd,next_36_months_dpd,total_enq_till_date,unique_enquiry_purpose,total_enq_amt,user_id,loan_id
96687,1315-0,9-20,2020-09-01,658.0,0,MALE,41,1,0,3,...,6,0,0,0,180,0,0,0,1315,0
96688,1315-0,10-20,2020-10-01,658.0,0,MALE,41,1,0,3,...,6,0,0,0,180,0,0,0,1315,0
96689,1315-0,11-20,2020-11-01,658.0,0,MALE,41,1,0,3,...,6,0,0,29,180,0,0,0,1315,0
96690,1315-0,12-20,2020-12-01,658.0,0,MALE,41,1,0,3,...,6,0,0,90,180,0,0,0,1315,0
96691,1315-0,1-21,2021-01-01,658.0,0,MALE,42,1,0,3,...,6,0,0,180,180,0,0,0,1315,0
96692,1315-0,2-21,2021-02-01,658.0,0,MALE,42,1,0,3,...,6,0,0,180,180,0,0,0,1315,0
96693,1315-0,3-21,2021-03-01,658.0,0,MALE,42,1,0,3,...,6,0,0,180,180,0,0,0,1315,0
96694,1315-0,4-21,2021-04-01,658.0,0,MALE,42,1,0,3,...,6,0,0,180,180,0,0,0,1315,0
96695,1315-0,5-21,2021-05-01,658.0,0,MALE,42,1,0,3,...,6,0,29,180,180,0,0,0,1315,0
96696,1315-0,6-21,2021-06-01,658.0,0,MALE,42,1,0,3,...,6,0,90,180,180,0,0,0,1315,0


In [92]:
def get_loan_type_ownership(row):
    row['loan_type'] = df_raw_dpd_new[(df_raw_dpd_new['user_id']==row['user_id']) & (df_raw_dpd_new['loan_id']==row['loan_id'])]['loan_type'].tolist()[0]
    row['ownership'] = df_raw_dpd_new[(df_raw_dpd_new['user_id']==row['user_id']) & (df_raw_dpd_new['loan_id']==row['loan_id'])]['ownership'].tolist()[0]
    return row

In [93]:
df_cibil_feature = df_cibil_feature.apply(get_loan_type_ownership,axis=1)

In [94]:
def get_enquiry_type(row):
    enq_list = df_enquiry[df_enquiry['user_id']==row['user_id']]['enq_purpose'].tolist()
    if len(enq_list)>0:
        
        row['enquiry_purpose'] = df_enquiry[df_enquiry['user_id']==row['user_id']]['enq_purpose'].tolist()[0]
    else:
        row['enquiry_purpose']=''
    return row

In [95]:
# def get_tot_enq_amt

In [196]:
# df_cibil_feature.head()

In [97]:
df_cibil_feature = df_cibil_feature.apply(get_enquiry_type,axis=1)

In [98]:
## l1 9-11 
## l1 10-11
## l1 

In [99]:
df_cibil_feature.columns

Index(['key', 'timestamp', 'datetime_formatted', 'cibil_score', 'total_email',
       'gender', 'age', 'open_loans', 'closed_loans', 'total_address',
       'sanc_amount', 'total_loans', 'total_phone_nos', 'dpd', 'dpd_bucket',
       'last_3_months_dpd', 'last_6_months_dpd', 'last_12_months_dpd',
       'last_36_months_dpd', 'next_3_months_dpd', 'next_6_months_dpd',
       'next_12_months_dpd', 'next_36_months_dpd', 'total_enq_till_date',
       'unique_enquiry_purpose', 'total_enq_amt', 'user_id', 'loan_id',
       'loan_type', 'ownership', 'enquiry_purpose'],
      dtype='object')

In [100]:
df_cibil_feature.dtypes

key                        object
timestamp                  object
datetime_formatted         object
cibil_score               float64
total_email                 int64
gender                     object
age                         int64
open_loans                  int64
closed_loans                int64
total_address               int64
sanc_amount               float64
total_loans                 int64
total_phone_nos             int64
dpd                        object
dpd_bucket                  int64
last_3_months_dpd           int64
last_6_months_dpd           int64
last_12_months_dpd          int64
last_36_months_dpd          int64
next_3_months_dpd           int64
next_6_months_dpd           int64
next_12_months_dpd          int64
next_36_months_dpd          int64
total_enq_till_date         int64
unique_enquiry_purpose      int64
total_enq_amt               int64
user_id                     int64
loan_id                     int64
loan_type                  object
ownership     

In [101]:
obj_cols = ['last_3_months_dpd', 'last_6_months_dpd', 'last_12_months_dpd',
       'last_36_months_dpd','next_3_months_dpd', 'next_6_months_dpd',
       'next_12_months_dpd', 'next_36_months_dpd']

In [102]:
for col in obj_cols:
    df_cibil_feature[col] = df_cibil_feature[col].apply(lambda x:float(x) if x is not None else 0)

In [103]:
test_cibil_feats = df_cibil_feature[['user_id','loan_type','datetime_formatted', 'last_3_months_dpd', 'last_6_months_dpd', 'last_12_months_dpd',
       'last_36_months_dpd','next_3_months_dpd', 'next_6_months_dpd',
       'next_12_months_dpd', 'next_36_months_dpd']].groupby(['user_id','loan_type','datetime_formatted']).sum().reset_index()

In [104]:
# test_cibil_feats

In [105]:
f_all = df_cibil_feature[['user_id','datetime_formatted', 'last_3_months_dpd', 'last_6_months_dpd', 'last_12_months_dpd',
       'last_36_months_dpd','next_3_months_dpd', 'next_6_months_dpd',
       'next_12_months_dpd', 'next_36_months_dpd']].groupby(['user_id','datetime_formatted']).sum().reset_index()

In [106]:
f_all

,user_id,datetime_formatted,last_3_months_dpd,last_6_months_dpd,last_12_months_dpd,last_36_months_dpd,next_3_months_dpd,next_6_months_dpd,next_12_months_dpd,next_36_months_dpd
0,0,2016-11-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,2016-12-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,2017-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,2017-02-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,2017-03-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
40987,1315,2021-12-01,29.0,29.0,29.0,35.0,90.0,90.0,90.0,90.0
40988,1315,2022-01-01,90.0,90.0,90.0,96.0,0.0,0.0,0.0,0.0
40989,1315,2022-02-01,180.0,180.0,180.0,186.0,0.0,0.0,0.0,0.0
40990,1315,2022-03-01,151.0,180.0,180.0,186.0,0.0,0.0,0.0,0.0


In [107]:
f_all.columns =['user_id','datetime_formatted', 'last_3_months_dpd_all', 'last_6_months_dpd_all', 'last_12_months_dpd_all',
       'last_36_months_dpd_all','next_3_months_dpd_all', 'next_6_months_dpd_all',
       'next_12_months_dpd_all', 'next_36_months_dpd_all']

In [108]:
df_cibil_feature = df_cibil_feature.merge(f_all,on=['user_id','datetime_formatted'],how='left')

In [109]:
# created f_all | f_gold_loan | f_cv | f_personal | f_credit_card 

In [110]:
# test_cibil_feats[test_cibil_feats['last_3_months_dpd']!=0]

In [111]:
test_cibil_feats.columns = ['user_id','loan_type','datetime_formatted', 'last_3_months_dpd_on_user_id_loan_type', 'last_6_months_dpd_on_user_id_loan_type', 'last_12_months_dpd_on_user_id_loan_type',
       'last_36_months_dpd_on_user_id_loan_type','next_3_months_dpd_on_user_id_loan_type', 'next_6_months_dpd_on_user_id_loan_type', 'next_12_months_dpd_on_user_id_loan_type',
       'next_36_months_dpd_on_user_id_loan_type']

In [112]:
test_cibil_feats['loan_type'].value_counts()

Commercial Vehicle Loan                    13311
Consumer Loan                               8165
Gold Loan                                   8118
Two-wheeler Loan                            6454
 Auto Loan                                  3593
Personal Loan                               3567
                                            3494
Business Loan Priority Sector               2824
Other                                       2308
Credit Card                                 2307
Auto Loan                                   1797
 Commercial Vehicle Loan                    1225
 Gold Loan                                  1161
 Consumer Loan                              1126
Business Loan General                        632
                                             630
Housing Loan                                 619
 Personal Loan                               574
 Business Loan Priority Sector               535
0 Gold Loan                                  303
 Two-wheeler Loan   

In [113]:
df_cibil_feature[(df_cibil_feature['loan_type']=='Gold Loan') & ( df_cibil_feature['dpd_bucket']>0)][['key','dpd']]

,key,dpd
85,3-6,032
135,3-6,010
139,3-6,402
180,3-2,017
183,3-6,017
...,...,...
95946,1307-4,200
95947,1307-4,231
95948,1307-4,259
96012,1308-13,188


In [193]:
test_cibil_feats.shape

(65061, 11)

In [115]:
# df_cibil_feature[df_cibil_feature['key']=='91-4']

In [116]:
df_cibil_feature.shape

(96707, 39)

In [117]:
df_cibil_feature = df_cibil_feature.merge(test_cibil_feats,on=['user_id','loan_type','datetime_formatted'],how='left')

In [118]:
test_enq_feats = df_cibil_feature[['user_id','enquiry_purpose','datetime_formatted', 'total_enq_till_date', 'unique_enquiry_purpose',
       'total_enq_amt']].groupby(['user_id','enquiry_purpose','datetime_formatted']).sum().reset_index()

In [119]:
test_enq_feats.head(1)

,user_id,enquiry_purpose,datetime_formatted,total_enq_till_date,unique_enquiry_purpose,total_enq_amt
0,0,,2016-11-01,0,0,0


In [120]:
test_enq_feats.columns = ['user_id','enquiry_purpose','datetime_formatted', 'total_enq_till_date_on_user_id_enq_purpose', 'unique_enquiry_purpose_on_user_id_enq_purpose',
       'total_enq_amt_on_user_id_enq_purpose']

In [121]:
df_cibil_feature = df_cibil_feature.merge(test_enq_feats,on=['user_id','enquiry_purpose','datetime_formatted'],how='left')

In [122]:
# df_cibil_feature.shape

In [123]:
feat_dict = {'Gold Loan' : 'gl', 
             'Personal Loan' : 'personal',
             'Commercial Vehicle Loan' : 'cvl',
             'Credit Card' : 'cc'
            }

In [124]:
def get_all_loan_type_feats(df,loan_type):
    df_l= df[df['loan_type']==loan_type] #'Gold Loan'
    
    final_cols = ['user_id','datetime_formatted']
    col_list = ['last_3_months_dpd', 'last_6_months_dpd', 'last_12_months_dpd',
       'last_36_months_dpd','next_3_months_dpd', 'next_6_months_dpd',
       'next_12_months_dpd', 'next_36_months_dpd']
    df_grp = df_l[['user_id','datetime_formatted', 'last_3_months_dpd', 'last_6_months_dpd', 'last_12_months_dpd',
       'last_36_months_dpd','next_3_months_dpd', 'next_6_months_dpd',
       'next_12_months_dpd', 'next_36_months_dpd']].groupby(['user_id','datetime_formatted']).sum().reset_index()
    key_to_add = feat_dict[loan_type]
    for col in col_list:
        temp = col + '_' +key_to_add
        final_cols.append(temp)
    df_grp.columns = final_cols
    df = df.merge(df_grp,on=['user_id','datetime_formatted'],how='left')
    return df

In [125]:
for item in list(feat_dict.keys()):
    df_cibil_feature= get_all_loan_type_feats(df_cibil_feature,item)

In [126]:
df_cibil_feature

,key,timestamp,datetime_formatted,cibil_score,total_email,gender,age,open_loans,closed_loans,total_address,...,next_12_months_dpd_cvl,next_36_months_dpd_cvl,last_3_months_dpd_cc,last_6_months_dpd_cc,last_12_months_dpd_cc,last_36_months_dpd_cc,next_3_months_dpd_cc,next_6_months_dpd_cc,next_12_months_dpd_cc,next_36_months_dpd_cc
0,0-10,11-16,2016-11-01,645.0,4,MALE,36,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0-10,12-16,2016-12-01,645.0,4,MALE,36,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0-10,1-17,2017-01-01,645.0,4,MALE,36,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0-10,2-17,2017-02-01,645.0,4,MALE,36,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0-10,3-17,2017-03-01,645.0,4,MALE,37,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96702,1315-0,12-21,2021-12-01,658.0,0,MALE,42,1,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96703,1315-0,1-22,2022-01-01,658.0,0,MALE,43,0,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96704,1315-0,2-22,2022-02-01,658.0,0,MALE,43,0,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96705,1315-0,3-22,2022-03-01,658.0,0,MALE,43,0,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [127]:
df_cibil_feature.to_csv('./dw_out/all_feat_cibil_pdf_v1.csv',index=False)

In [128]:
df_cibil_feature = pd.read_csv('./dw_out/all_feat_cibil_pdf_v1.csv')

/home/ubuntu/anaconda3/envs/faiss/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (30) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# V1 model - XGBoost | Loan Type Feats

In [129]:
df_cibil_feat = pd.read_csv('./dw_out/all_feat_cibil_pdf_v1.csv')

In [130]:
df_cibil_feat.shape

(96707, 82)

In [131]:
df_cibil_feat.user_id.nunique()

847

In [132]:
df_cibil_feat[['user_id','loan_id']].drop_duplicates()

,user_id,loan_id
0,0,10
23,0,9
25,0,8
52,0,4
80,3,6
...,...,...
96590,1313,6
96600,1313,5
96620,1313,2
96630,1313,0


In [133]:
# 2436-337

In [134]:
df_cibil_feat.columns

Index(['key', 'timestamp', 'datetime_formatted', 'cibil_score', 'total_email',
       'gender', 'age', 'open_loans', 'closed_loans', 'total_address',
       'sanc_amount', 'total_loans', 'total_phone_nos', 'dpd', 'dpd_bucket',
       'last_3_months_dpd', 'last_6_months_dpd', 'last_12_months_dpd',
       'last_36_months_dpd', 'next_3_months_dpd', 'next_6_months_dpd',
       'next_12_months_dpd', 'next_36_months_dpd', 'total_enq_till_date',
       'unique_enquiry_purpose', 'total_enq_amt', 'user_id', 'loan_id',
       'loan_type', 'ownership', 'enquiry_purpose', 'last_3_months_dpd_all',
       'last_6_months_dpd_all', 'last_12_months_dpd_all',
       'last_36_months_dpd_all', 'next_3_months_dpd_all',
       'next_6_months_dpd_all', 'next_12_months_dpd_all',
       'next_36_months_dpd_all', 'last_3_months_dpd_on_user_id_loan_type',
       'last_6_months_dpd_on_user_id_loan_type',
       'last_12_months_dpd_on_user_id_loan_type',
       'last_36_months_dpd_on_user_id_loan_type',
       'ne

In [135]:
# df_cibil_feat.sort_values(['user_id','loan_id','datetime_formatted'])[:15]

In [136]:
# df_cibil_feat.sort_values(['user_id','loan_id','datetime_formatted'],inplace=True)

In [137]:
# df_cibil_feat['open_loans_prev'] = df_cibil_feat['open_loans'].shift(1)

In [138]:
# df_cibil_feat['open_loans_prev'].fillna(0,inplace=True)

In [139]:
# df_cibil_feat['user_id_prev'] = df_cibil_feat['user_id'].shift(1)

In [140]:
# df_cibil_feat[['user_id','loan_id','closed_date']][:20]

In [141]:
df_cibil_feat['key'].nunique()

6254

In [142]:
# df_cibil_feat[df_cibil_feat['key']=='0-1']

In [143]:
# 0-1 , 0-11 > 0-2

In [144]:
# df_cibil_feat1 =  df_cibil_feat[(df_cibil_feat['open_loans_prev']!=df_cibil_feat['open_loans']) & (df_cibil_feat['total_loans']>1)]

In [145]:
# df_cibil_feat.groupby('key').agg(lambda x: x.tolist())


In [146]:
# df_cibil_feat1[['key','open_loans','closed_loans','total_loans']][:20]

In [147]:
# df_cibil_feat.sort_values('key',inplace=True)

In [148]:
## user X loan -> remove first loan based on timestamp (loan_open_date)

In [149]:
df_cibil_feat

,key,timestamp,datetime_formatted,cibil_score,total_email,gender,age,open_loans,closed_loans,total_address,...,next_12_months_dpd_cvl,next_36_months_dpd_cvl,last_3_months_dpd_cc,last_6_months_dpd_cc,last_12_months_dpd_cc,last_36_months_dpd_cc,next_3_months_dpd_cc,next_6_months_dpd_cc,next_12_months_dpd_cc,next_36_months_dpd_cc
0,0-10,11-16,2016-11-01,645.0,4,MALE,36,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0-10,12-16,2016-12-01,645.0,4,MALE,36,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0-10,1-17,2017-01-01,645.0,4,MALE,36,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0-10,2-17,2017-02-01,645.0,4,MALE,36,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0-10,3-17,2017-03-01,645.0,4,MALE,37,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96702,1315-0,12-21,2021-12-01,658.0,0,MALE,42,1,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96703,1315-0,1-22,2022-01-01,658.0,0,MALE,43,0,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96704,1315-0,2-22,2022-02-01,658.0,0,MALE,43,0,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
96705,1315-0,3-22,2022-03-01,658.0,0,MALE,43,0,1,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [150]:
# df_cibil_feat[['user_id','loan_id']].drop_duplicates()

In [151]:
df_cibil_feat['datetime_formatted'] = pd.to_datetime(df_cibil_feat.datetime_formatted, errors = 'coerce')

first_loan_df = df_cibil_feat.groupby(['user_id', 'loan_id']).agg({'datetime_formatted': 'min'}).reset_index()
first_loan_df['time_rank'] = df_cibil_feat.groupby(['user_id'])['datetime_formatted'].rank('dense', ascending=True)
first_loan_df = first_loan_df[first_loan_df.time_rank !=1]

df_cibil_feat = pd.merge(df_cibil_feat, first_loan_df, how = 'inner') 

In [152]:
# df_cibil_feat[df_cibil_feat[]]

In [153]:
# first_loan_df = df_cibil_feat.sort_values(['user_id','datetime_formatted']).groupby('user_id').agg({'loan_id': lambda x: list(x)[0]
#                                  }).reset_index()

In [154]:
df_cibil_feat

,key,timestamp,datetime_formatted,cibil_score,total_email,gender,age,open_loans,closed_loans,total_address,...,next_36_months_dpd_cvl,last_3_months_dpd_cc,last_6_months_dpd_cc,last_12_months_dpd_cc,last_36_months_dpd_cc,next_3_months_dpd_cc,next_6_months_dpd_cc,next_12_months_dpd_cc,next_36_months_dpd_cc,time_rank
0,0-10,11-16,2016-11-01,645.0,4,MALE,36,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
1,0-9,10-18,2018-10-01,645.0,4,MALE,38,1,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
2,0-8,11-18,2018-11-01,645.0,4,MALE,38,2,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,3-6,3-18,2018-03-01,757.0,1,FEMALE,24,1,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
4,3-4,9-18,2018-09-01,757.0,1,FEMALE,25,3,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6190,1313-6,6-16,2016-06-01,654.0,2,MALE,40,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0
6191,1313-5,1-19,2019-01-01,654.0,2,MALE,42,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0
6192,1313-2,9-20,2020-09-01,654.0,2,MALE,44,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0
6193,1313-0,2-21,2021-02-01,654.0,2,MALE,44,3,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0


In [155]:
def grouper_df(df,key='key'):
    
    ## idea is to remove first loan  based on timestamp 
    
    
#     df = df[df['total_loans']>1]
#     dfs =[]
#     for i in range(92):
#         new_df = df[df['user_id']==i]
#         remove_loan_id = fist_loan_df[fist_loan_df['user_id']==i]['loan_id'].tolist()[0]
#         new_df = new_df[new_df['loan_id'] !=remove_loan_id]
#         dfs.append(new_df)
#     df=pd.concat(dfs)
# #     df = df[df['loan_id']!=0]
# #     df.reset_index(inplace=True)
    col_list = list(df.columns)
    col_list.remove('key')
    
    print(col_list)
    obj = df.groupby('key')
    new_df = pd.DataFrame()
    new_df['key'] = obj['key'].first()
    for col in col_list:
        new_df[col] = obj[col].apply(lambda x:list(x)[0])
    new_df.index = range(len(new_df))
    new_df.sort_values(['user_id','loan_id'],inplace=True)
    return new_df

In [156]:
# df_new = grouper_df(df_cibil_feat)

In [157]:
# df_new[df_new['user_id']==90]

In [158]:
# df_cibil_feat.sort_values(['user_id','datetime_formatted'])[:20]

In [159]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

## convert feats to numeric

In [160]:
# df_cibil_feat[df_cibil_feat['key']=='0-5']
# u0-0
# u0- 0 kth 
# u0 -1 (1st change point)


In [161]:
# df_new

In [162]:
df_cibil_feat

,key,timestamp,datetime_formatted,cibil_score,total_email,gender,age,open_loans,closed_loans,total_address,...,next_36_months_dpd_cvl,last_3_months_dpd_cc,last_6_months_dpd_cc,last_12_months_dpd_cc,last_36_months_dpd_cc,next_3_months_dpd_cc,next_6_months_dpd_cc,next_12_months_dpd_cc,next_36_months_dpd_cc,time_rank
0,0-10,11-16,2016-11-01,645.0,4,MALE,36,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
1,0-9,10-18,2018-10-01,645.0,4,MALE,38,1,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
2,0-8,11-18,2018-11-01,645.0,4,MALE,38,2,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,3-6,3-18,2018-03-01,757.0,1,FEMALE,24,1,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
4,3-4,9-18,2018-09-01,757.0,1,FEMALE,25,3,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6190,1313-6,6-16,2016-06-01,654.0,2,MALE,40,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0
6191,1313-5,1-19,2019-01-01,654.0,2,MALE,42,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0
6192,1313-2,9-20,2020-09-01,654.0,2,MALE,44,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0
6193,1313-0,2-21,2021-02-01,654.0,2,MALE,44,3,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0


In [163]:
df_cibil_feat.dtypes

key                              object
timestamp                        object
datetime_formatted       datetime64[ns]
cibil_score                     float64
total_email                       int64
                              ...      
next_3_months_dpd_cc            float64
next_6_months_dpd_cc            float64
next_12_months_dpd_cc           float64
next_36_months_dpd_cc           float64
time_rank                       float64
Length: 83, dtype: object

In [164]:
# df_cibil_feat['gender'].describe()

## change points 

## avg time series of dpd on user level 

In [165]:
df_cibil_feat['gender'] = df_cibil_feat['gender'].apply(lambda x: 1 if x=='Male' else 0)

In [166]:
def get_loan_type_encoder(loan_type):
    if loan_type=='Commercial Vehicle Loan':
        return 5
    elif loan_type=='Consumer Loan':
        return 4
    elif loan_type=='Gold Loan':
        return 3
    elif loan_type=='Personal Loan':
        return 2
    elif loan_type=='Credit Card':
        return 1
    else:
        return 0

In [167]:
list_loan_types = list(df_cibil_feat['loan_type'].unique())

In [168]:
for item in list_loan_types:
    print(item,df_cibil_feat[df_cibil_feat['loan_type']==item].shape[0]*100/df_cibil_feat.shape[0])

Two-wheeler Loan 4.471347861178369
Consumer Loan 15.221953188054883
Gold Loan 29.37853107344633
Other 4.051654560129136
Personal Loan 9.798224374495561
Commercial Vehicle Loan 13.656174334140436
nan 0.0
 Two-wheeler Loan 0.16142050040355124
 Consumer Loan 0.9846650524616626
 Personal Loan 1.0653753026634383
Credit Card 1.7433414043583535
 Commercial Vehicle Loan 0.8071025020177562
Business Loan Priority Sector 4.858757062146893
 Gold Loan 1.937046004842615
  0.6618240516545602
 Business Loan Priority Sector 0.9685230024213075
 Auto Loan 2.437449556093624
0 Gold Loan 0.4519774011299435
Business Loan General 0.41969330104923325
0 Consumer Loan 0.048426150121065374
Auto Loan 1.2267958030669894
Overdraft 0.258272800645682
Loan Against Bank Deposits 0.30669895076674736
 Loan Against Bank Deposits 0.08071025020177562
Aut Loan 0.016142050040355124
 Credit Card 0.016142050040355124
 Overdraft 0.08071025020177562
Housing Loan 0.4519774011299435
 Housing Loan 0.11299435028248588
Property Loan 0.

In [169]:
df_cibil_feat['loan_type'] = df_cibil_feat['loan_type'].apply(get_loan_type_encoder)

In [170]:
df_cibil_feat['ownership'].unique()

array([' JOINT', ' INDIVIDUAL', ' GUARANTOR', nan, 'JOINT',
       ' AUTHORISED USER', '- JOINT'], dtype=object)

In [171]:
for item in list(df_cibil_feat['ownership'].unique()):
    print(item,df_cibil_feat[df_cibil_feat['ownership']==item].shape[0]*100/df_cibil_feat.shape[0])

 JOINT 7.796610169491525
 INDIVIDUAL 87.44148506860371
 GUARANTOR 4.697336561743342
nan 0.0
JOINT 0.016142050040355124
 AUTHORISED USER 0.016142050040355124
- JOINT 0.016142050040355124


In [172]:
def get_ownership_encoder(ownership):
    if ownership=='Individual':
        return 3
    elif ownership=='Guarantor':
        return 2
    else:
        return 1

In [173]:
df_cibil_feat['ownership'] = df_cibil_feat['ownership'].apply(get_ownership_encoder)

In [174]:
df_cibil_feat['enquiry_purpose'].unique()

array([nan, 'Credit Card', 'Commercial Vehicle Loan',
       'Loan Against Shares / Security', 'Two-wheeler Loan', 'Other',
       'Auto Loan', 'Personal Loan', 'Housing Loan', 'Consumer Loan',
       'Gold Loan', 'Business Loan General', 'Property Loan',
       'Microfinance-Others'], dtype=object)

In [175]:
for item in list(df_cibil_feat['enquiry_purpose'].unique()):
    print(item,df_cibil_feat[df_cibil_feat['enquiry_purpose']==item].shape[0]*100/df_cibil_feat.shape[0])

nan 0.0
Credit Card 1.6464891041162228
Commercial Vehicle Loan 8.248587570621469
Loan Against Shares / Security 0.387409200968523
Two-wheeler Loan 2.066182405165456
Other 1.8886198547215496
Auto Loan 2.050040355125101
Personal Loan 1.9854721549636805
Housing Loan 0.46811945117029863
Consumer Loan 2.437449556093624
Gold Loan 0.08071025020177562
Business Loan General 0.048426150121065374
Property Loan 0.0645682001614205
Microfinance-Others 0.16142050040355124


In [176]:
def get_enquiry_purpose_encoder(enquiry_purpose):
    if enquiry_purpose=='Commercial Vehicle Loan':
        return 5
    elif enquiry_purpose=='Auto Loan (Personal)':
        return 4
    elif enquiry_purpose=='Tractor Loan':
        return 3
    elif enquiry_purpose=='Business Loan – General':
        return 2
    else:
        return 1

In [177]:
df_cibil_feat['enquiry_purpose'] = df_cibil_feat['enquiry_purpose'].apply(get_enquiry_purpose_encoder)

## training and test split 

In [178]:
## 14 pdfs 
## 


In [179]:
## 619 pdf | ~ 1week   
## some 569 training pdf | (38 k user * loan )
##(2.4k change points)
## -> 

In [180]:
# df_cibil_feat = df_cibil_feat[df_cibil_feat['loan_type']>4]

In [181]:
df_cibil_feat

,key,timestamp,datetime_formatted,cibil_score,total_email,gender,age,open_loans,closed_loans,total_address,...,next_36_months_dpd_cvl,last_3_months_dpd_cc,last_6_months_dpd_cc,last_12_months_dpd_cc,last_36_months_dpd_cc,next_3_months_dpd_cc,next_6_months_dpd_cc,next_12_months_dpd_cc,next_36_months_dpd_cc,time_rank
0,0-10,11-16,2016-11-01,645.0,4,0,36,1,0,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0
1,0-9,10-18,2018-10-01,645.0,4,0,38,1,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0
2,0-8,11-18,2018-11-01,645.0,4,0,38,2,1,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,3-6,3-18,2018-03-01,757.0,1,0,24,1,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0
4,3-4,9-18,2018-09-01,757.0,1,0,25,3,0,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6190,1313-6,6-16,2016-06-01,654.0,2,0,40,1,1,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0
6191,1313-5,1-19,2019-01-01,654.0,2,0,42,1,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,102.0
6192,1313-2,9-20,2020-09-01,654.0,2,0,44,2,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0
6193,1313-0,2-21,2021-02-01,654.0,2,0,44,3,2,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101.0


In [182]:
# df_nested_list

In [183]:
# len(df_nested_list['AccountInformation'][0])

In [184]:
## remove age and total from feats | relative age from date 
## 

In [185]:
# df_ci

In [186]:
df_cibil_feat.iloc[8]

key                                      3-1
timestamp                               2-21
datetime_formatted       2021-02-01 00:00:00
cibil_score                            757.0
total_email                                1
                                ...         
next_3_months_dpd_cc                     NaN
next_6_months_dpd_cc                     NaN
next_12_months_dpd_cc                    NaN
next_36_months_dpd_cc                    NaN
time_rank                                5.0
Name: 8, Length: 83, dtype: object

In [187]:
def next_3_month_bucet(next_3_months_dpd_on_user_id_loan_type):
    if next_3_months_dpd_on_user_id_loan_type==0:
        return 0
    elif next_3_months_dpd_on_user_id_loan_type>0 and next_3_months_dpd_on_user_id_loan_type <=30:
        return 1
    elif next_3_months_dpd_on_user_id_loan_type>30 and next_3_months_dpd_on_user_id_loan_type<=90:
        return 2
    else:
        return 3

In [188]:
df_cibil_feat['output'] = df_cibil_feat['next_6_months_dpd'].apply(lambda x:next_3_month_bucet(x))

In [189]:
df_cibil_feat['next_6_months_dpd'].describe()

count    6195.000000
mean       30.551090
std       204.765656
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max      5400.000000
Name: next_6_months_dpd, dtype: float64

In [190]:
df_cibil_feat['output'].value_counts()

0    5400
3     386
1     224
2     185
Name: output, dtype: int64

In [197]:
df_cibil_feat.columns

Index(['key', 'timestamp', 'datetime_formatted', 'cibil_score', 'total_email',
       'gender', 'age', 'open_loans', 'closed_loans', 'total_address',
       'sanc_amount', 'total_loans', 'total_phone_nos', 'dpd', 'dpd_bucket',
       'last_3_months_dpd', 'last_6_months_dpd', 'last_12_months_dpd',
       'last_36_months_dpd', 'next_3_months_dpd', 'next_6_months_dpd',
       'next_12_months_dpd', 'next_36_months_dpd', 'total_enq_till_date',
       'unique_enquiry_purpose', 'total_enq_amt', 'user_id', 'loan_id',
       'loan_type', 'ownership', 'enquiry_purpose', 'last_3_months_dpd_all',
       'last_6_months_dpd_all', 'last_12_months_dpd_all',
       'last_36_months_dpd_all', 'next_3_months_dpd_all',
       'next_6_months_dpd_all', 'next_12_months_dpd_all',
       'next_36_months_dpd_all', 'last_3_months_dpd_on_user_id_loan_type',
       'last_6_months_dpd_on_user_id_loan_type',
       'last_12_months_dpd_on_user_id_loan_type',
       'last_36_months_dpd_on_user_id_loan_type',
       'ne

In [192]:
df_cibil_feat.to_csv("./dw_out/df_cibil_feat.csv")

In [189]:
X = df_cibil_feat[['cibil_score', 'total_email',
       'gender', 'age', 'open_loans', 'closed_loans', 'total_address','sanc_amount',
        'total_phone_nos', 'total_enq_till_date', 'unique_enquiry_purpose',
       'total_enq_amt',  'loan_type', 'ownership',
       'enquiry_purpose','last_3_months_dpd_gl',
       'last_6_months_dpd_gl', 'last_12_months_dpd_gl',
       'last_36_months_dpd_gl','last_3_months_dpd_personal', 'last_6_months_dpd_personal',
       'last_12_months_dpd_personal', 'last_36_months_dpd_personal','last_3_months_dpd_cvl', 'last_6_months_dpd_cvl',
       'last_12_months_dpd_cvl', 'last_36_months_dpd_cvl','last_3_months_dpd_cc', 'last_6_months_dpd_cc', 'last_12_months_dpd_cc',
       'last_36_months_dpd_cc',
       'total_enq_till_date_on_user_id_enq_purpose',
       'unique_enquiry_purpose_on_user_id_enq_purpose',
       'total_enq_amt_on_user_id_enq_purpose']]

In [190]:
# 

In [191]:
# df_cibil_feat['dpd_bucket']

In [192]:
# df

In [193]:
y= df_cibil_feat[['output']]

In [214]:
X.to_csv("./dw_out/pd_data_X.csv")

In [215]:
y.to_csv("./dw_out/pd_data_y.csv")

In [194]:
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split


In [195]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)


In [196]:
from sklearn.metrics import DistanceMetric


In [197]:
# import imblearn
# print(imblearn.__version__)

In [198]:
y['output'].value_counts()

0    2125
3     142
2      74
1      72
Name: output, dtype: int64

In [199]:
# from imblearn.over_sampling import RandomOverSampler
 

In [200]:
X_train.shape, X_test.shape

((1930, 34), (483, 34))

In [201]:
# y_tr
# ain

In [202]:
y_train['output'].value_counts()

0    1704
3     111
1      61
2      54
Name: output, dtype: int64

In [203]:
xgb_classifier =XGBClassifier(n_estimators=1000)


In [204]:
from sklearn.utils.class_weight import compute_sample_weight
sample_weights = compute_sample_weight(
    class_weight='balanced',
    y=y_train['output'] #provide your own target name
)
# 
# xgb_classifier.fit(X, y, sample_weight=sample_weights)

In [205]:
xgb_classifier.fit(X_train,y_train,sample_weight=sample_weights)


/home/ubuntu/anaconda3/envs/faiss/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/ubuntu/anaconda3/envs/faiss/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/ubuntu/anaconda3/envs/faiss/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example 

[05:38:22] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=1000, n_jobs=2,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [206]:
predictions = xgb_classifier.predict(X_test)


In [207]:
y_proba = xgb_classifier.predict_proba(X_test)

In [208]:
print("Accuracy of Model::",accuracy_score(y_test,predictions))


Accuracy of Model:: 0.8530020703933747


In [209]:
y_test['output'].value_counts()

0    421
3     31
2     20
1     11
Name: output, dtype: int64

In [210]:
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve,classification_report

def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """

    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

In [211]:
print(confusion_matrix(y_test,predictions))

[[397  10   4  10]
 [  7   2   1   1]
 [ 13   2   2   3]
 [ 18   2   0  11]]


In [217]:
print(classification_report(y_test,predictions))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93       421
           1       0.12      0.18      0.15        11
           2       0.29      0.10      0.15        20
           3       0.44      0.35      0.39        31

    accuracy                           0.85       483
   macro avg       0.44      0.39      0.40       483
weighted avg       0.84      0.85      0.84       483



In [213]:
import pickle
file_name = "./dw_out/xgb_cibil_classifier.pkl"

# save
pickle.dump(xgb_classifier, open(file_name, "wb"))


In [220]:
# load
xgb_model_loaded = pickle.load(open(file_name, "rb"))


In [222]:
# !cp xgb_cibil_classifier.pkl ../model_dir/

In [237]:
# xgb_model_loaded.predict(X_test)